BERT+CRF模型训练，finetuning

In [1]:
# 显卡查看
! nvidia-smi

Thu May 21 02:58:58 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.82       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   64C    P0    32W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [2]:
# 依赖安装
! pip install fastNLP

加载数据集

In [3]:
import sys

from fastNLP.core import Const
from fastNLP.io import PeopleDailyNERLoader
from fastNLP.io import PeopleDailyPipe

sys.path.insert(0, '/content/drive/My Drive/my_framework/qyt_clue/')  # 定义搜索路径的优先顺序，序号从0开始，表示最大优先级

import myClue  # noqa
print('myClue module path :{}'.format(myClue.__file__))  # 输出测试模块文件位置
from myClue.core import logger  # noqa
from myClue.core.utils import print_data_bundle  # noqa
from myClue.tools.serialize import save_serialize_obj  # noqa

myClue module path :/content/drive/My Drive/my_framework/qyt_clue/myClue/__init__.py


In [4]:
train_file_config = {
    'train': '/content/drive/My Drive/my_framework/qyt_clue/data/weibo_NER/train.conll',
    'dev': '/content/drive/My Drive/my_framework/qyt_clue/data/weibo_NER/dev.conll',
    'test': '/content/drive/My Drive/my_framework/qyt_clue/data/weibo_NER/test.conll',
}
# train_data_bundle_pkl_file = './data/weibo_NER/train_data_bundle.pkl'
logger.info('数据加载')
data_loader = PeopleDailyNERLoader()
data_bundle = data_loader.load(train_file_config)
print_data_bundle(data_bundle)
logger.info('数据预处理')
data_pipe = PeopleDailyPipe()
data_bundle = data_pipe.process(data_bundle)
data_bundle.rename_field(field_name=Const.CHAR_INPUT, new_field_name=Const.INPUT, ignore_miss_dataset=True, rename_vocab=True)
print_data_bundle(data_bundle)
# save_serialize_obj(data_bundle, train_data_bundle_pkl_file)
# logger.info('数据预处理后进行序列化：{}'.format(train_data_bundle_pkl_file))

2020-05-21 02:59:06 I [<ipython-input-4-ed08f1d37e0d>:7] 数据加载
2020-05-21 02:59:06 I [utils.py:16] dataset name : train
2020-05-21 02:59:06 I [utils.py:17] dataset len : 1350
2020-05-21 02:59:06 I [utils.py:18] dataset example : 
2020-05-21 02:59:06 I [utils.py:19] 
+------------------------------------------+------------------------------------------+
| raw_chars                                | target                                   |
+------------------------------------------+------------------------------------------+
| ['科', '技', '全', '方', '位', '资', ... | ['O', 'O', 'O', 'O', 'O', 'O', 'O', '... |
| ['对', '，', '输', '给', '一', '个', ... | ['O', 'O', 'O', 'O', 'O', 'O', 'B-PER... |
| ['今', '天', '下', '午', '起', '来', ... | ['O', 'O', 'O', 'O', 'O', 'O', 'O', '... |
| ['今', '年', '拜', '年', '不', '短', ... | ['O', 'O', 'O', 'O', 'O', 'O', 'O', '... |
| ['浑', '身', '酸', '疼', '，', '两', ... | ['O', 'O', 'O', 'O', 'O', 'O', 'O', '... |
+------------------------------------------+----------------

+-------------+-----------+--------+
| field_names | raw_chars | target |
+-------------+-----------+--------+
|   is_input  |   False   | False  |
|  is_target  |   False   | False  |
| ignore_type |           |        |
|  pad_value  |           |        |
+-------------+-----------+--------+
+-------------+-----------+--------+
| field_names | raw_chars | target |
+-------------+-----------+--------+
|   is_input  |   False   | False  |
|  is_target  |   False   | False  |
| ignore_type |           |        |
|  pad_value  |           |        |
+-------------+-----------+--------+
+-------------+-----------+--------+
| field_names | raw_chars | target |
+-------------+-----------+--------+
|   is_input  |   False   | False  |
|  is_target  |   False   | False  |
| ignore_type |           |        |
|  pad_value  |           |        |
+-------------+-----------+--------+


2020-05-21 02:59:07 I [utils.py:16] dataset name : train
2020-05-21 02:59:07 I [utils.py:17] dataset len : 1350
2020-05-21 02:59:07 I [utils.py:18] dataset example : 
2020-05-21 02:59:07 I [utils.py:19] 
+------------------------+------------------------+---------+------------------------+
| raw_chars              | target                 | seq_len | words                  |
+------------------------+------------------------+---------+------------------------+
| ['科', '技', '全', ... | [0, 0, 0, 0, 0, 0, ... | 26      | [792, 1015, 156, 19... |
| ['对', '，', '输', ... | [0, 0, 0, 0, 0, 0, ... | 15      | [123, 2, 1205, 115,... |
| ['今', '天', '下', ... | [0, 0, 0, 0, 0, 0, ... | 79      | [164, 33, 65, 739, ... |
| ['今', '年', '拜', ... | [0, 0, 0, 0, 0, 0, ... | 18      | [164, 92, 880, 92, ... |
| ['浑', '身', '酸', ... | [0, 0, 0, 0, 0, 0, ... | 18      | [2072, 221, 1056, 9... |
+------------------------+------------------------+---------+------------------------+
2020-05-21 02:59:07 I [utils

+-------------+-----------+--------+---------+-------+
| field_names | raw_chars | target | seq_len | words |
+-------------+-----------+--------+---------+-------+
|   is_input  |   False   |  True  |   True  |  True |
|  is_target  |   False   |  True  |   True  | False |
| ignore_type |           | False  |  False  | False |
|  pad_value  |           |   0    |    0    |   0   |
+-------------+-----------+--------+---------+-------+
+-------------+-----------+--------+---------+-------+
| field_names | raw_chars | target | seq_len | words |
+-------------+-----------+--------+---------+-------+
|   is_input  |   False   |  True  |   True  |  True |
|  is_target  |   False   |  True  |   True  | False |
| ignore_type |           | False  |  False  | False |
|  pad_value  |           |   0    |    0    |   0   |
+-------------+-----------+--------+---------+-------+
+-------------+-----------+--------+---------+-------+
| field_names | raw_chars | target | seq_len | words |
+---------

模型训练

In [0]:
import os
import sys
import time

import torch

from fastNLP import Const
from fastNLP.core.callback import GradientClipCallback
from fastNLP.core.callback import WarmupCallback
from fastNLP import SpanFPreRecMetric
from fastNLP import Trainer
from fastNLP import LossInForward
from fastNLP.core.optimizer import AdamW
from fastNLP.embeddings import BertEmbedding
from fastNLP import Tester

from myClue.core import logger  # noqa
from myClue.core.callback import EarlyStopCallback  # noqa
from myClue.models.bert_crf import BertCRF  # noqa
from myClue.tools.serialize import load_serialize_obj  # noqa
from myClue.tools.serialize import save_serialize_obj  # noqa
from myClue.tools.file import init_file_path  # noqa

In [6]:
# train_data_bundle_pkl_file = '/content/drive/My Drive/my_framework/qyt_clue/data/weibo_NER/train_data_bundle.pkl'
model_path = '/content/drive/My Drive/my_framework/qyt_clue/data/weibo_NER/model_bert_fine_tuning'
init_file_path(model_path)
logger.add_file_handler(os.path.join(model_path, 'log_{}.txt'.format(time.strftime("%Y-%m-%d-%H-%M-%S", time.localtime()))))  # 日志写入文件
char_vocab_pkl_file = os.path.join(model_path, 'vocab_char.pkl')
target_vocab_pkl_file = os.path.join(model_path, 'target_char.pkl')
# logger.warn('加载数据集')
# data_bundle = load_serialize_obj(train_data_bundle_pkl_file)
logger.warn('获取词典')
char_vocab = data_bundle.get_vocab('words')
logger.info('char_vocab:{}'.format(char_vocab))
target_vocab = data_bundle.get_vocab('target')
logger.info('target_vocab:{}'.format(target_vocab))
save_serialize_obj(char_vocab, char_vocab_pkl_file)
save_serialize_obj(target_vocab, target_vocab_pkl_file)
logger.info('词典序列化:{}'.format(char_vocab_pkl_file))
logger.warn('选择预训练词向量')
model_dir_or_name = '/content/drive/My Drive/data/bert-chinese-wwm'
# model_dir_or_name = 'cn-wwm-ext'
bert_embed = BertEmbedding(vocab=char_vocab, model_dir_or_name=model_dir_or_name, pool_method='max', requires_grad=True, layers='11', include_cls_sep=False, dropout=0.5, word_dropout=0.01, auto_truncate=True)
logger.warn('神经网络模型')
model = BertCRF(bert_embed, tag_vocab=target_vocab, encoding_type='bio')
logger.info(model)
logger.warn('训练超参数设定')
loss = LossInForward()
optimizer = AdamW([param for param in model.parameters() if param.requires_grad], lr=2e-5)
# metric = AccuracyMetric()
metric = SpanFPreRecMetric(tag_vocab=data_bundle.get_vocab(Const.TARGET), only_gross=False)  # 若only_gross=False, 即还会返回各个label的metric统计值
device = 'cuda' if torch.cuda.is_available() else 'cpu'  # 如果有gpu的话在gpu上运行，训练速度会更快
logger.info('device:{}'.format(device))

2020-05-21 02:59:07 W [<ipython-input-6-b17940fbd25e>:9] 获取词典
2020-05-21 02:59:07 I [<ipython-input-6-b17940fbd25e>:11] char_vocab:Vocabulary(['科', '技', '全', '方', '位']...)
2020-05-21 02:59:07 I [<ipython-input-6-b17940fbd25e>:13] target_vocab:Vocabulary(['O', 'B-PER.NOM', 'I-PER.NOM', 'B-LOC.NAM', 'I-LOC.NAM']...)
2020-05-21 02:59:07 I [<ipython-input-6-b17940fbd25e>:16] 词典序列化:/content/drive/My Drive/my_framework/qyt_clue/data/weibo_NER/model_bert_fine_tuning/vocab_char.pkl
2020-05-21 02:59:07 W [<ipython-input-6-b17940fbd25e>:17] 选择预训练词向量
/usr/local/lib/python3.6/dist-packages/fastNLP/embeddings/bert_embedding.py:99: UserWarning: For Chinese bert, pooled_method should choose from 'first', 'last' in order to achieve faster speed.
  warnings.warn("For Chinese bert, pooled_method should choose from 'first', 'last' in order to achieve"


For Chinese bert, pooled_method should choose from 'first', 'last' in order to achieve faster speed.
loading vocabulary file /root/.fastNLP/embedding/bert-chinese-wwm-ext/vocab.txt


2020-05-21 02:59:09 W [<ipython-input-6-b17940fbd25e>:21] 神经网络模型
2020-05-21 02:59:09 I [<ipython-input-6-b17940fbd25e>:23] BertCRF(
  (embed): BertEmbedding(
    (dropout_layer): Dropout(p=0.5, inplace=False)
    (model): _BertWordModel(
      (encoder): BertModel(
        (embeddings): BertEmbeddings(
          (word_embeddings): Embedding(3405, 768)
          (position_embeddings): Embedding(512, 768)
          (token_type_embeddings): Embedding(2, 768)
          (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (encoder): BertEncoder(
          (layer): ModuleList(
            (0): BertLayer(
              (attention): BertAttention(
                (self): BertSelfAttention(
                  (query): Linear(in_features=768, out_features=768, bias=True)
                  (key): Linear(in_features=768, out_features=768, bias=True)
                  (value): Linear(in_features=768, out_features=768,

Load pre-trained BERT parameters from file /root/.fastNLP/embedding/bert-chinese-wwm-ext/pytorch_model.bin.
Start to generate word pieces for word.
87 words are unsegmented. Among them, 51 added to the BPE vocab.


In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
# 模型训练
batch_size = 32
n_epochs = 200
early_stopping = 10
callbacks = [GradientClipCallback(clip_type='norm', clip_value=1),
              WarmupCallback(warmup=0.1, schedule='linear'),
              EarlyStopCallback(early_stopping)]
trainer = Trainer(
    save_path=model_path,
    train_data=data_bundle.get_dataset('train'),
    model=model,
    loss=loss,
    optimizer=optimizer,
    batch_size=batch_size,
    n_epochs=n_epochs,
    dev_data=data_bundle.get_dataset('dev'),
    metrics=metric,
    metric_key='f',
    device=device,
    callbacks=callbacks)
logger.warn('开始训练')
trainer.train()

input fields after batch(if batch size is 2):
	target: (1)type:torch.Tensor (2)dtype:torch.int64, (3)shape:torch.Size([2, 26]) 
	seq_len: (1)type:torch.Tensor (2)dtype:torch.int64, (3)shape:torch.Size([2]) 
	words: (1)type:torch.Tensor (2)dtype:torch.int64, (3)shape:torch.Size([2, 26]) 
target fields after batch(if batch size is 2):
	target: (1)type:torch.Tensor (2)dtype:torch.int64, (3)shape:torch.Size([2, 26]) 
	seq_len: (1)type:torch.Tensor (2)dtype:torch.int64, (3)shape:torch.Size([2]) 



2020-05-21 02:59:13 W [<ipython-input-8-1b877b80695f>:21] 开始训练


training epochs started 2020-05-21-02-59-13-338611


/pytorch/torch/csrc/utils/python_arg_parser.cpp:756: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha)


Evaluate data in 3.46 seconds!


...

2020-05-21 03:17:26 W [callback.py:38] ======epoch : 19 , early stopping : 9/10======
2020-05-21 03:17:26 I [callback.py:40] metric_key : f, metric_value : 0.712531
2020-05-21 03:17:26 I [callback.py:41] eval_result : 
{'SpanFPreRecMetric': {'f-gpe.nam': 0.677419, 'pre-gpe.nam': 0.583333, 'rec-gpe.nam': 0.807692, 'f-gpe.nom': 0.0, 'pre-gpe.nom': 0.0, 'rec-gpe.nom': 0.0, 'f-loc.nam': 0.769231, 'pre-loc.nam': 0.714286, 'rec-loc.nam': 0.833333, 'f-loc.nom': 0.5, 'pre-loc.nom': 0.5, 'rec-loc.nom': 0.5, 'f-per.nom': 0.762353, 'pre-per.nom': 0.746544, 'rec-per.nom': 0.778846, 'f-org.nom': 0.428571, 'pre-org.nom': 0.333333, 'rec-org.nom': 0.6, 'f-per.nam': 0.806452, 'pre-per.nam': 0.78125, 'rec-per.nam': 0.833333, 'f-org.nam': 0.415842, 'pre-org.nam': 0.388889, 'rec-org.nam': 0.446809, 'f': 0.712531, 'pre': 0.682353, 'rec': 0.745501}}


Evaluate data in 3.53 seconds!
Evaluation on dev at Epoch 19/200. Step:817/8600: 
SpanFPreRecMetric: f-gpe.nam=0.677419, pre-gpe.nam=0.583333, rec-gpe.nam=0.807692, f-gpe.nom=0.0, pre-gpe.nom=0.0, rec-gpe.nom=0.0, f-loc.nam=0.769231, pre-loc.nam=0.714286, rec-loc.nam=0.833333, f-loc.nom=0.5, pre-loc.nom=0.5, rec-loc.nom=0.5, f-per.nom=0.762353, pre-per.nom=0.746544, rec-per.nom=0.778846, f-org.nom=0.428571, pre-org.nom=0.333333, rec-org.nom=0.6, f-per.nam=0.806452, pre-per.nam=0.78125, rec-per.nam=0.833333, f-org.nam=0.415842, pre-org.nam=0.388889, rec-org.nam=0.446809, f=0.712531, pre=0.682353, rec=0.745501



2020-05-21 03:18:22 W [callback.py:38] ======epoch : 20 , early stopping : 10/10======
2020-05-21 03:18:22 I [callback.py:40] metric_key : f, metric_value : 0.706161
2020-05-21 03:18:22 I [callback.py:41] eval_result : 
{'SpanFPreRecMetric': {'f-gpe.nam': 0.754098, 'pre-gpe.nam': 0.657143, 'rec-gpe.nam': 0.884615, 'f-gpe.nom': 0.0, 'pre-gpe.nom': 0.0, 'rec-gpe.nom': 0.0, 'f-loc.nam': 0.714286, 'pre-loc.nam': 0.625, 'rec-loc.nam': 0.833333, 'f-loc.nom': 0.4, 'pre-loc.nom': 0.333333, 'rec-loc.nom': 0.5, 'f-per.nom': 0.776256, 'pre-per.nom': 0.73913, 'rec-per.nom': 0.817308, 'f-org.nom': 0.5, 'pre-org.nom': 0.428571, 'rec-org.nom': 0.6, 'f-per.nam': 0.7343, 'pre-per.nam': 0.649573, 'rec-per.nam': 0.844444, 'f-org.nam': 0.375, 'pre-org.nam': 0.367347, 'rec-org.nam': 0.382979, 'f': 0.706161, 'pre': 0.654945, 'rec': 0.766067}}
2020-05-21 03:18:22 I [callback.py:46] reach early stopping patience, stop training.
2020-05-21 03:18:22 I [callback.py:55] Early Stopping triggered in epoch 20!


Evaluate data in 3.49 seconds!
Reloaded the best model.

In Epoch:9/Step:387, got best dev performance:
SpanFPreRecMetric: f-gpe.nam=0.727273, pre-gpe.nam=0.6, rec-gpe.nam=0.923077, f-gpe.nom=0.0, pre-gpe.nom=0.0, rec-gpe.nom=0.0, f-loc.nam=0.333333, pre-loc.nam=0.25, rec-loc.nam=0.5, f-loc.nom=0.25, pre-loc.nom=0.5, rec-loc.nom=0.166667, f-per.nom=0.782805, pre-per.nom=0.739316, rec-per.nom=0.831731, f-org.nom=0.888889, pre-org.nom=1.0, rec-org.nom=0.8, f-per.nam=0.748718, pre-per.nam=0.695238, rec-per.nam=0.811111, f-org.nam=0.372093, pre-org.nam=0.410256, rec-org.nam=0.340426, f=0.712727, pre=0.674312, rec=0.755784


{'best_epoch': 9,
 'best_eval': {'SpanFPreRecMetric': {'f': 0.712727,
   'f-gpe.nam': 0.727273,
   'f-gpe.nom': 0.0,
   'f-loc.nam': 0.333333,
   'f-loc.nom': 0.25,
   'f-org.nam': 0.372093,
   'f-org.nom': 0.888889,
   'f-per.nam': 0.748718,
   'f-per.nom': 0.782805,
   'pre': 0.674312,
   'pre-gpe.nam': 0.6,
   'pre-gpe.nom': 0.0,
   'pre-loc.nam': 0.25,
   'pre-loc.nom': 0.5,
   'pre-org.nam': 0.410256,
   'pre-org.nom': 1.0,
   'pre-per.nam': 0.695238,
   'pre-per.nom': 0.739316,
   'rec': 0.755784,
   'rec-gpe.nam': 0.923077,
   'rec-gpe.nom': 0.0,
   'rec-loc.nam': 0.5,
   'rec-loc.nom': 0.166667,
   'rec-org.nam': 0.340426,
   'rec-org.nom': 0.8,
   'rec-per.nam': 0.811111,
   'rec-per.nom': 0.831731}},
 'best_step': 387,
 'seconds': 1150.4}

In [9]:
# 评测
logger.warn('训练后评估')
tester = Tester(data=data_bundle.get_dataset('test'), model=model, metrics=metric, batch_size=64, device=device)
test_result = tester.test()
logger.info('评估结果：\n{}'.format(test_result))

2020-05-21 03:18:23 W [<ipython-input-9-2a8ad2ebd772>:2] 训练后评估


2020-05-21 03:18:27 I [<ipython-input-9-2a8ad2ebd772>:5] 评估结果：
{'SpanFPreRecMetric': {'f-gpe.nam': 0.841121, 'pre-gpe.nam': 0.75, 'rec-gpe.nam': 0.957447, 'f-gpe.nom': 0.0, 'pre-gpe.nom': 0.0, 'rec-gpe.nom': 0.0, 'f-loc.nam': 0.387097, 'pre-loc.nam': 0.5, 'rec-loc.nam': 0.315789, 'f-loc.nom': 0.333333, 'pre-loc.nom': 0.666667, 'rec-loc.nom': 0.222222, 'f-per.nom': 0.709677, 'pre-per.nom': 0.66, 'rec-per.nom': 0.767442, 'f-org.nom': 0.3, 'pre-org.nom': 1.0, 'rec-org.nom': 0.176471, 'f-per.nam': 0.697095, 'pre-per.nam': 0.65625, 'rec-per.nam': 0.743363, 'f-org.nam': 0.506329, 'pre-org.nam': 0.5, 'rec-org.nam': 0.512821, 'f': 0.675926, 'pre': 0.654709, 'rec': 0.698565}}


Evaluate data in 3.26 seconds!
[tester] 
SpanFPreRecMetric: f-gpe.nam=0.841121, pre-gpe.nam=0.75, rec-gpe.nam=0.957447, f-gpe.nom=0.0, pre-gpe.nom=0.0, rec-gpe.nom=0.0, f-loc.nam=0.387097, pre-loc.nam=0.5, rec-loc.nam=0.315789, f-loc.nom=0.333333, pre-loc.nom=0.666667, rec-loc.nom=0.222222, f-per.nom=0.709677, pre-per.nom=0.66, rec-per.nom=0.767442, f-org.nom=0.3, pre-org.nom=1.0, rec-org.nom=0.176471, f-per.nam=0.697095, pre-per.nam=0.65625, rec-per.nam=0.743363, f-org.nam=0.506329, pre-org.nam=0.5, rec-org.nam=0.512821, f=0.675926, pre=0.654709, rec=0.698565
